<h3>model of thrombin inhibition with varied AT</h3>

At 4 nM thrombin and 1400 nM thrombin.

In [ ]:
using DifferentialEquations
using ODEInterfaceDiffEq
using Catalyst
using Latexify
using PyPlot
using DataFrames
using XLSX

In [ ]:
versioninfo()

In [ ]:
model = @reaction_network begin
    k_AT, IIa + AT --> IIaAT
    k_HC, IIa + HC --> IIaHC
    k_a1PI, IIa + a1PI --> IIaa1PI
    k_a2M, IIa + a2M --> IIaa2M
end

In [ ]:
ode_model = convert(ODESystem, model)

In [ ]:
paramsmap(ode_model)

In [ ]:
# Second order rate constants for thrombin inhibition in M-1 sec-1
#
k2_AT = 10000.0 # M-1 sec-1
k2_a2M = 3900.0; # M-1 sec-1
k2_HC = 2700.0 # M-1 sec-1
k2_a1PI = 16.0; # M-1 sec-1

In [ ]:
# model uses all concentrations in nM
#
IIa = 4.0 # nM
#
AT = 3500.0 # nM
HC = 1200.0 # nM
a1PI = 32000.0 # nM
a2M = 2400.0; # nM

In [ ]:
# solve the system for thrombin inactivation with varied concentrations of AT
#
IIa_4nM_AT_only = Dict{String,Any}() # create the file for storing the output of the model
#
con_AT = [1.0, 0.3, 0.15, 0.09, 0.03, 0.0] # fraction of initial AT concentration
#
for i in con_AT
    function solve_model(;
            # the rate constants need to be converted to nM-1 sec-1 
            # to match the concentrations of reactants
            k_HC = k2_HC/1000000000, # nM-1 min-1
            k_a1PI = k2_a1PI/1000000000, # nM-1 min-1
            k_AT = k2_AT/1000000000, # nM-1 min-1
            k_a2M = k2_a2M/1000000000, # nM-1 min-1
        
            IIa_t0 = IIa, # nM
            AT_t0 = i*AT, # nM
            HC_t0 = 0.0, # nM
            a1PI_t0 = 0.0, # nM
            a2M_t0 = 0.0, # nM
            
            IIaAT_t0 = 0.0,
            IIaHC_t0 = 0.0,
            IIaa1PI_t0 = 0.0,
            IIaa2M_t0 = 0.0
            )
        
            p = [k_HC, k_a1PI, k_AT, k_a2M] # needs to match the order in the paramsmap
            u0 = [IIa_t0, AT_t0, IIaAT_t0, HC_t0, IIaHC_t0, a1PI_t0, IIaa1PI_t0, a2M_t0, IIaa2M_t0]
            tspan = (0.0,2400.0)
            prob = ODEProblem(model, u0, tspan, p)
            sol=solve(prob)
            return sol
    end
    out = solve_model();  # output is [column,row (time in this case)] # semicolon suppresses output
    #
    out_temp2 = zeros(length(out.t),1+size(out)[1])
    out_temp1 = Float64[]
    for i=1:length(out.t)
        out_temp1=(out.t[i],out[1,i],out[2,i],out[3,i],out[4,i],out[5,i],out[6,i],out[7,i],out[8,i],out[9,i])
        out_temp2[i,1]=out_temp1[1]
        for j=1:size(out)[1]
            out_temp2[i,j+1]=out_temp1[j+1]
        end
    end
    tag="AT $(round(Int, 100*i))%"
    names=["time", "IIa", "AT", "IIaAT", "HC", "IIaHC", "a1PI", "IIaa1PI", "a2M", "IIaa2M"]
    IIa_4nM_AT_only[tag] = DataFrame(out_temp2, names)
end

In [ ]:
tag_names = Array{String}(undef,length(IIa_4nM_AT_only))
for i in 1:length(con_AT)
    tag_names[i]="AT $(round(Int, 100*con_AT[i]))%"
end

In [ ]:
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
tick_params(axis="both", which="both", direction="in", length=5, width=1.5)
plt.ylabel("[free thrombin] (nM)", fontsize = 18)
plt.xlabel("time (seconds)", fontsize = 18)
for i in 1:length(tag_names)
    plot(IIa_4nM_AT_only[tag_names[i]][!,"time"], 
        IIa_4nM_AT_only[tag_names[i]][!,"IIa"], 
        label=tag_names[i])
end
plt.legend(loc="upper right", reverse="True")

In [ ]:
# solve the system for thrombin inactivation with varied concentrations of AT
#
IIa_4nM = Dict{String,Any}() # create the file for storing the output of the model
#
con_AT = [1.0, 0.3, 0.15, 0.09, 0.03, 0.0] # fraction of initial AT concentration
#
for i in con_AT
    function solve_model(;
            # the rate constants need to be converted to nM-1 sec-1 
            # to match the concentrations of reactants
            k_HC = k2_HC/1000000000, # nM-1 min-1
            k_a1PI = k2_a1PI/1000000000, # nM-1 min-1
            k_AT = k2_AT/1000000000, # nM-1 min-1
            k_a2M = k2_a2M/1000000000, # nM-1 min-1
        
            IIa_t0 = IIa, # nM
            AT_t0 = i*AT, # nM
            HC_t0 = HC, # nM
            a1PI_t0 = a1PI, # nM
            a2M_t0 = a2M, # nM
            
            IIaAT_t0 = 0.0,
            IIaHC_t0 = 0.0,
            IIaa1PI_t0 = 0.0,
            IIaa2M_t0 = 0.0
            )
        
            p = [k_HC, k_a1PI, k_AT, k_a2M] # needs to match the order in the paramsmap
            u0 = [IIa_t0, AT_t0, IIaAT_t0, HC_t0, IIaHC_t0, a1PI_t0, IIaa1PI_t0, a2M_t0, IIaa2M_t0]
            tspan = (0.0,600.0)
            prob = ODEProblem(model, u0, tspan, p)
            sol=solve(prob)
            return sol
    end
    out = solve_model();  # output is [column,row (time in this case)] # semicolon suppresses output
    #
    out_temp2 = zeros(length(out.t),1+size(out)[1])
    out_temp1 = Float64[]
    for i=1:length(out.t)
        out_temp1=(out.t[i],out[1,i],out[2,i],out[3,i],out[4,i],out[5,i],out[6,i],out[7,i],out[8,i],out[9,i])
        out_temp2[i,1]=out_temp1[1]
        for j=1:size(out)[1]
            out_temp2[i,j+1]=out_temp1[j+1]
        end
    end
    tag="AT $(round(Int, 100*i))%"
    names=["time", "IIa", "AT", "IIaAT", "HC", "IIaHC", "a1PI", "IIaa1PI", "a2M", "IIaa2M"]
    IIa_4nM[tag] = DataFrame(out_temp2, names)
end

In [ ]:
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
tick_params(axis="both", which="both", direction="in", length=5, width=1.5)
plt.ylabel("[free thrombin] (nM)", fontsize = 18)
plt.xlabel("time (seconds)", fontsize = 18)
for i in 1:length(tag_names)
    plot(IIa_4nM[tag_names[i]][!,"time"], 
        IIa_4nM[tag_names[i]][!,"IIa"], 
        label=tag_names[i])
end
plt.legend(loc="upper right", reverse="True")

In [ ]:
# model uses all concentrations in nM
#
IIa = 1400.0 # nM
#
# solve the system for thrombin inactivation with varied concentrations of AT
#
IIa_1400nM = Dict{String,Any}() # create the file for storing the output of the model
#
con_AT = [1.0, 0.3, 0.15, 0.09, 0.03, 0.0] # fraction of initial AT concentration
#
for i in con_AT
    function solve_model(;
            # the rate constants need to be converted to nM-1 sec-1 
            # to match the concentrations of reactants
            k_HC = k2_HC/1000000000, # nM-1 min-1
            k_a1PI = k2_a1PI/1000000000, # nM-1 min-1
            k_AT = k2_AT/1000000000, # nM-1 min-1
            k_a2M = k2_a2M/1000000000, # nM-1 min-1
        
            IIa_t0 = IIa, # nM
            AT_t0 = i*AT, # nM
            HC_t0 = HC, # nM
            a1PI_t0 = a1PI, # nM
            a2M_t0 = a2M, # nM
            
            IIaAT_t0 = 0.0,
            IIaHC_t0 = 0.0,
            IIaa1PI_t0 = 0.0,
            IIaa2M_t0 = 0.0
            )
        
            p = [k_HC, k_a1PI, k_AT, k_a2M] # needs to match the order in the paramsmap
            u0 = [IIa_t0, AT_t0, IIaAT_t0, HC_t0, IIaHC_t0, a1PI_t0, IIaa1PI_t0, a2M_t0, IIaa2M_t0]
            tspan = (0.0,600.0)
            prob = ODEProblem(model, u0, tspan, p)
            sol=solve(prob)
            return sol
    end
    out = solve_model();  # output is [column,row (time in this case)] # semicolon suppresses output
    #
    out_temp2 = zeros(length(out.t),1+size(out)[1])
    out_temp1 = Float64[]
    for i=1:length(out.t)
        out_temp1=(out.t[i],out[1,i],out[2,i],out[3,i],out[4,i],out[5,i],out[6,i],out[7,i],out[8,i],out[9,i])
        out_temp2[i,1]=out_temp1[1]
        for j=1:size(out)[1]
            out_temp2[i,j+1]=out_temp1[j+1]
        end
    end
    tag="AT $(round(Int, 100*i))%"
    names=["time", "IIa", "AT", "IIaAT", "HC", "IIaHC", "a1PI", "IIaa1PI", "a2M", "IIaa2M"]
    IIa_1400nM[tag] = DataFrame(out_temp2, names)
end

In [ ]:
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
tick_params(axis="both", which="both", direction="in", length=5, width=1.5)
# plt.ylim(-6,119.0)
plt.ylabel("[free thrombin] (nM)", fontsize = 18)
plt.xlabel("time (seconds)", fontsize = 18)
# legend locations - upper right, upper left, lower left, lower right, right, center left, 
#                    center right, lower center, upper center, center
# plt.savefig("output_name.eps")
for i in 1:length(tag_names)
    plot(IIa_1400nM[tag_names[i]][!,"time"], 
        IIa_1400nM[tag_names[i]][!,"IIa"], 
        label=tag_names[i])
end
plt.legend(loc="upper right", reverse="True")

In [ ]:
df = XLSX.openxlsx("IIa_4nM_AT_only_ODE.xlsx", mode="w") # open a new excel file
#
i=0
for sheetname in tag_names
    i=i+1
    if i==1
        XLSX.rename!(df[1], sheetname)
        XLSX.writetable!(df[1], convert(DataFrame, IIa_4nM_AT_only[sheetname]))
    else
        XLSX.addsheet!(df, sheetname)
        XLSX.writetable!(df[sheetname], convert(DataFrame, IIa_4nM_AT_only[sheetname]))      
    end
end
#
XLSX.writexlsx("IIa_4nM_AT_only_ODE.xlsx", df, overwrite=true)

In [ ]:
df = XLSX.openxlsx("IIa_4nM_ODE.xlsx", mode="w") # open a new excel file
#
i=0
for sheetname in tag_names
    i=i+1
    if i==1
        XLSX.rename!(df[1], sheetname)
        XLSX.writetable!(df[1], convert(DataFrame, IIa_4nM[sheetname]))
    else
        XLSX.addsheet!(df, sheetname)
        XLSX.writetable!(df[sheetname], convert(DataFrame, IIa_4nM[sheetname]))      
    end
end
#
XLSX.writexlsx("IIa_4nM_ODE.xlsx", df, overwrite=true)

In [ ]:
df = XLSX.openxlsx("IIa_1400nM_ODE.xlsx", mode="w") # open a new excel file
#
i=0
for sheetname in tag_names
    i=i+1
    if i==1
        XLSX.rename!(df[1], sheetname)
        XLSX.writetable!(df[1], convert(DataFrame, IIa_1400nM[sheetname]))
    else
        XLSX.addsheet!(df, sheetname)
        XLSX.writetable!(df[sheetname], convert(DataFrame, IIa_1400nM[sheetname]))      
    end
end
#
XLSX.writexlsx("IIa_1400nM_ODE.xlsx", df, overwrite=true)

In [ ]:
row_names = String["IIa (nM)","AT (nM)","IIaAT (nM)","HC (nM)",
    "IIaHC (nM)","a1PI (nM)","IIaa1PI (nM)","A2M (nM)","IIaA2M (nM)"];
column_names = ["100% AT", "30% AT", "15% AT", "9% AT", "3% AT", "no AT"];
#
fig1 = figure("4 nM IIa", figsize=(15,15))
ax1 = plt.subplot(9,6,1)
subplots_adjust(hspace=0.0)
subplots_adjust(wspace=0.0)
for i in 1:9
    for j in 1:length(tag_names)
        if j==1
            plt.subplot(9,6,j+6*(i-1))
            plot(IIa_4nM[tag_names[j]][!,"time"], 
                IIa_4nM[tag_names[j]][!,i+1])
            ax1=gca()
        else
            plt.subplot(9,6,j+6*(i-1), 
                sharey=plt.subplot(9,6,1+6*(i-1)))
            plot(IIa_4nM[tag_names[j]][!,"time"], 
                IIa_4nM[tag_names[j]][!,i+1])
            ax1=gca()
            ax1.tick_params(labelleft=false)
        end
    end
end
#
# title the columns
# 
for i=1:6
    ax1=subplot(9,6,i)
    ax1.set_title(column_names[i], size=14)
end
# 
# label the rows
#
for i=1:9
    plt.subplot(9,6,1+6*(i-1))
    plt.ylabel(row_names[i], size=12)
end
#
# set axis
#
for i=[1,13, 49]
    ax1=subplot(9,6,i)
    ax1.set_ylim([-0.5,4.5])
end
# 
for i=19
    ax1=subplot(9,6,i)
    ax1.set_ylim([1194.5,1200.5])
end
# 
for i=31
    ax1=subplot(9,6,i)
    ax1.set_ylim([31977,32005])
end
#
for i=43
    ax1=subplot(9,6,i)
    ax1.set_ylim([2395, 2401])
end
#
for i=49:54
    ax1=subplot(9,6,i)
    plt.xlabel("time (sec)", size=14)
end
#
fig1.suptitle("4 nM thrombin", size=24, y=0.93)

In [ ]:
fig2 = figure("1400 nM IIa", figsize=(15,15))
ax2 = plt.subplot(9,6,1)
subplots_adjust(hspace=0.0)
subplots_adjust(wspace=0.0)
for i in 1:9
    for j in 1:length(tag_names)
        if j==1
            plt.subplot(9,6,j+6*(i-1))
            plot(IIa_1400nM[tag_names[j]][!,"time"], 
                IIa_1400nM[tag_names[j]][!,i+1])
            ax1=gca()
        else
            plt.subplot(9,6,j+6*(i-1), 
                sharey=plt.subplot(9,6,1+6*(i-1)))
            plot(IIa_1400nM[tag_names[j]][!,"time"], 
                IIa_1400nM[tag_names[j]][!,i+1])
            ax1=gca()
            ax1.tick_params(labelleft=false)
        end
    end
end
#
# title the columns
# 
for i=1:6
    ax2=subplot(9,6,i)
    ax2.set_title(column_names[i], size=14)
end
# 
# label the rows
#
for i=1:9
    plt.subplot(9,6,1+6*(i-1))
    plt.ylabel(row_names[i], size=12)
end
#
# set axis
#
for i=49:54
    ax2=subplot(9,6,i)
    plt.xlabel("time (sec)", size=14)
end
#
fig2.suptitle("1400 nM thrombin", size=24, y=0.93)

In [ ]:
# fig1.savefig("IIa4nM.svg")
# fig2.savefig("IIa1400nM.svg")